# GO-NOGO_table
****
##説明

与えられた保安範囲について, 範囲内に落下する条件を表形式で自動出力するプログラム.
****
##ユーザへ

左側（デフォルト）にあるファイルタブから予め保安範囲などのデータをアップロードする.
複数指定された場合はいずれか一つの領域に含まれる場合はGOを出力する.


ファイルは一行目の緯度, 経度に対応する列に**"longitude", "latitude"** ないし**"経度", "緯度"**と書かれていること.
座標が2つ以下の場合は無視される.

具体的な落下点が書かれたcsvファイルは実行時に纏めてアップロードする.

結果はGO_NOGO.csvに保安範囲内に含まれる場合は1を, そうでない場合は0を表形式で纏めて出力される.

##下のセルを初めに必ず実行してください
自作クラス定義

In [ ]:
import geopandas as gpd     # 緯度・経度処理
from shapely.geometry import Point, Polygon# gpdの座標
import pandas as pd         # csvファイル
import numpy as np
from pathlib import Path    # パス
import shutil               # ディレクトリ削除
import os
import time                 # タイムスタンプ用
from google.colab import files # ファイルアップロード, ダウンロード

# 分散表示クラス
class scatter_map:
    wind_dir_name = ["北", "北北東", "北東", "東北東", "東", "東南東", "南東", "南南東", "南", "南南西", "南西", "西南西", "西", ]
    def __init__(self):
        self.launch_site_gpds = [] # 射場に関するDataFrame

    # 射場に関するcsvファイルを読み取る
    # file_names : ファイル名 (パス)
    def read_launch_site_data(self, file_names):
        temp_dfs = list(map(pd.read_csv, file_names))# DataFrame読み取り
        for df, name in zip(temp_dfs, file_names):
            if not ({"longitude", "latitude"} <= set(df.columns) or {"経度", "緯度"} <= set(df.columns)):
                continue
            if len(df) < 3:
                continue
            df.rename(columns={ "経度" : "longitude", "緯度" : "latitude" }, inplace=True) # "longitude", "latitude"に統一
            self.launch_site_gpds += [gpd.GeoDataFrame(crs="epsg:6668", geometry=[Polygon(df[["longitude", "latitude"]])])]

    # DataFrameの中から経度, 緯度 or longitude, latitudeを探し, 対応するペアを1組ずつ返す.
    def __read_scatter_data_pair(df):
        for col in df.columns:
            if "longitude" in col and col.replace("longitude", "latitude") in df.columns:
                yield df[[col, col.replace("longitude", "latitude")]] # longitude, latitudeの順で格納して返す

    # 落下分散のDataFrameと風速, 風向, 経度, 緯度に対応する列名
    def go_nogo_table(self, file_names, col_wind_speed, col_wind_dir, show_result = True, DL_result = True):
        if not len(file_names): return # ファイル入力がない場合
        output_dir = Path(f"GO_NOGO_RESULT_{int(time.time())}")
        os.mkdir(output_dir)
        for df, name in zip(map(pd.read_csv, file_names), map(Path, file_names)): # DataFrame読み取り
            os.mkdir(output_dir/name.stem) # ファイル名でディレクトリ作成
            for pair in scatter_map.__read_scatter_data_pair(df): # 経度・緯度のペアを1組ずつ読み取り
                coord = [gpd.GeoDataFrame(crs="epsg:6668", geometry=[Point(pair.iat[idx, 0], pair.iat[idx, 1])]) for idx in range(len(pair))]
                is_contained = pd.DataFrame([1 if any([gpd["geometry"][0].contains([coord[idx]])for gpd in self.launch_site_gpds]) else 0
                           for idx in range(len(pair))]) # すべてパターンについて, 与えられたジオメトリ領域に内包されているか
                if show_result:
                    display(pd.concat([df[[col_wind_speed, col_wind_dir]], is_contained], axis=1))
                if DL_result:
                    csv_name = pair.columns[0].replace("longitude", "") + ".csv"
                    df = pd.concat([df[[col_wind_speed, col_wind_dir]], is_contained], axis=1).sort_values([col_wind_speed, col_wind_dir]) # 必要なデータのみを抽出 & 内包されているかを追加
                    df.to_csv(output_dir/name.stem/csv_name) # save as csv
                    print(df)
                    df2 = pd.concat([df[df[col_wind_speed] == wind_speed].iloc[:, 1:3].set_index(col_wind_dir) for wind_speed in set(df[col_wind_speed])], axis=1, ignore_index=True)
                    df2.columns = set(df[col_wind_speed])
                    df2.to_csv(output_dir/name.stem/"GO_NOGO.csv")

        # DL用にZIPファイルに圧縮
        if DL_result: shutil.make_archive(output_dir,  format='zip', root_dir=output_dir)
        shutil.rmtree(output_dir) # ディレクトリごと削除
        if DL_result: files.download(output_dir.with_suffix(".zip")) # ファイルダウンロード

## メインコード

In [ ]:
import glob # ファイル構造
import warnings # 警告無視用（任意）
warnings.simplefilter("ignore", UserWarning) # 特定の警告を無視
%unload_ext google.colab.data_table

def main():
    # アップロードするデータの風速，風向，緯度，経度の列名
    col_wind_speed = "wind_speed[m/s]"
    col_wind_dir   = "wind_dir[deg]"

    map = scatter_map()
    map.read_launch_site_data(["落下可能域.csv"])
    # # ファイルアップロード
    file_names = list(files.upload().keys()) # 複数ファイルをいっきにUPしてもOK
    # file_names = ["para_with_detachment.csv"]
    map.go_nogo_table(file_names, col_wind_speed, col_wind_dir, show_result=False)
    for f in file_names: os.remove(f) # 射場データ以外のcsvファイルは削除

if __name__ == "__main__":
    main()

# 作成者
佐藤空馬 (14期)